In [ ]:
### building baseling model using Linear regression on the energy_consumption dataset

In [45]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor
import numpy as np

In [84]:

energy_consumption = pd.read_csv("energy_consumption.csv")
base_station_basic_info = pd.read_csv("./base_station_basic_info.csv")
cell_level_data = pd.read_csv("cell_level_data.csv")

/tmp/ipykernel_5290/705852802.py:3: DtypeWarning: Columns (3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  cell_level_data = pd.read_csv("cell_level_data.csv")


In [4]:
energy_consumption.head()

,Time,BS,Energy
0,1/1/2023 1:00,B_0,64.27503737
1,1/1/2023 2:00,B_0,55.90433483
2,1/1/2023 3:00,B_0,57.6980568
3,1/1/2023 4:00,B_0,55.15695067
4,1/1/2023 5:00,B_0,56.05381166


In [85]:
energy_consumption.rename(columns = {'Time':'Date-Time'}, inplace = True)

In [86]:
##breaking down Date-Time field
energy_consumption = energy_consumption[energy_consumption["Date-Time"].apply(lambda x: len(x.split(" ")) == 2)]
energy_consumption["Time"] = energy_consumption["Date-Time"].apply(lambda x : x.split(" ")[1])
energy_consumption["Date"] = energy_consumption["Date-Time"].apply(lambda x : x.split(" ")[0])
energy_consumption["Month"] = energy_consumption["Date"].apply(lambda x: int(x.split("/")[0]))
energy_consumption["Day"] = energy_consumption["Date"].apply(lambda x: int(x.split("/")[1]))
energy_consumption = energy_consumption.drop(columns=["Date","Date-Time"])
energy_consumption.head()


,BS,Energy,Time,Month,Day
0,B_0,64.27503737,1:00,1,1
1,B_0,55.90433483,2:00,1,1
2,B_0,57.6980568,3:00,1,1
3,B_0,55.15695067,4:00,1,1
4,B_0,56.05381166,5:00,1,1


In [87]:
##convert Time and Base Station to usable Values
energy_consumption["Time"] = energy_consumption["Time"].apply(lambda x : int(x.split(":")[0]))
energy_consumption["BS"] = energy_consumption["BS"].apply(lambda x : int(x.split("_")[1]))

In [91]:
#add base_station_info 

base_station_basic_info = base_station_basic_info[base_station_basic_info["BS"].apply(lambda x: len(x.split("_")) == 2)]
base_station_basic_info["BS"] = base_station_basic_info["BS"].apply(lambda x : int(x.split("_")[1]))
merged = pd.merge(energy_consumption,base_station_basic_info,on="BS")  

In [ ]:
merged.head()

In [8]:
##set categorical types
energy_consumption["Month"] = energy_consumption["Month"].astype("category")


In [9]:
##seperate label data and split train test data
energy_consumption_label = energy_consumption["Energy"]
energy_consumption = energy_consumption.drop(columns=["Energy"])

train_x,test_x,train_y,test_y = train_test_split(energy_consumption,energy_consumption_label,test_size=0.2)



In [70]:
xg_reg = xgb.XGBRegressor(objective='reg:squarederror',  n_estimators=100,  max_depth = 10, enable_categorical=True,learning_rate=0.1)

In [71]:
xg_reg.fit(train_x,train_y)

/home/abundance/.local/share/virtualenvs/zindi-5g-energy-consumption-vq-XQ_S3/lib/python3.11/site-packages/xgboost/data.py:463: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  or is_sparse(dtype)
/home/abundance/.local/share/virtualenvs/zindi-5g-energy-consumption-vq-XQ_S3/lib/python3.11/site-packages/xgboost/data.py:464: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  or (is_categorical_dtype(dtype) and enable_categorical)
/home/abundance/.local/share/virtualenvs/zindi-5g-energy-consumption-vq-XQ_S3/lib/python3.11/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/abundance/.local/share/virtualenvs/zindi-5g-energy-consumption-vq-XQ_S3/lib/python3.11/site-packages/xgboo

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [72]:
preds = xg_reg.predict(test_x)
rmse = np.sqrt(mean_squared_error(test_y, preds))
print("RMSE: %f" % (rmse))

RMSE: 10.972426


/home/abundance/.local/share/virtualenvs/zindi-5g-energy-consumption-vq-XQ_S3/lib/python3.11/site-packages/xgboost/data.py:463: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  or is_sparse(dtype)
/home/abundance/.local/share/virtualenvs/zindi-5g-energy-consumption-vq-XQ_S3/lib/python3.11/site-packages/xgboost/data.py:464: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  or (is_categorical_dtype(dtype) and enable_categorical)
/home/abundance/.local/share/virtualenvs/zindi-5g-energy-consumption-vq-XQ_S3/lib/python3.11/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/abundance/.local/share/virtualenvs/zindi-5g-energy-consumption-vq-XQ_S3/lib/python3.11/site-packages/xgboo

In [82]:
params = {'loss_function':'RMSE', # objective function
          'eval_metric':'AUC', # metric
          'verbose': 200, # output to stdout info about training process every 200 iterations
          'random_seed': 123,
          'iterations':2000,
          'learning_rate':0.1
         }

cbc_1 = CatBoostRegressor(**params,cat_features=["Month"])
cbc_1.fit(train_x,train_y)

0:	total: 4.84ms	remaining: 9.69s
200:	total: 1.08s	remaining: 9.63s
400:	total: 2.25s	remaining: 8.96s
600:	total: 3.42s	remaining: 7.97s
800:	total: 4.6s	remaining: 6.89s
1000:	total: 5.76s	remaining: 5.75s
1200:	total: 7.03s	remaining: 4.68s
1400:	total: 8.21s	remaining: 3.51s
1600:	total: 9.41s	remaining: 2.35s
1800:	total: 10.6s	remaining: 1.17s
1999:	total: 11.8s	remaining: 0us


In [83]:

preds = cbc_1.predict(test_x)
rmse = np.sqrt(mean_squared_error(test_y, preds))
print("RMSE: %f" % (rmse))

RMSE: 10.961078
